In [1]:
# import the following modules

import requests
import pandas as pd
import getpass
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

print ('modules loaded')

modules loaded


In [2]:
# Log in to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on  getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    # Prompt the user to enter their AGOL username
    username = input("Enter your ArcGIS Online username: ")

    # Prompt the user to enter their AGOL password (without echoing)
    password = getpass.getpass("Enter your ArcGIS Online password: ")

    # Create a GIS object and log in to ArcGIS Online
    gis = GIS("https://www.arcgis.com", username, password)

    # Return the GIS object
    return gis

# Example usage:
gis = login_AGOL()
print("Logged in as:", gis.users.me.username)

Enter your ArcGIS Online username: stacey_vernon_build
Enter your ArcGIS Online password: ········
Logged in as: stacey_vernon_build


In [3]:
# documentation on using ArcGIS API to create feature sevice from GeoJSON here: https://developers.arcgis.com/python/guide/import-data/
# this is a "static" GeoJSON file for the Dublin Bike Locatins and the Dublin Bike purple zone

# set up the feature service properties in AGOL for the stations and zone dataset
item_properties = {
    "title": "BikeStations_Zones",  # Provide a title for your item
    "description": "Dublin Bike Stations and Zones from GeoJSON", # add a description to your data
    "tags": "dublinBikes", # add tags to enhance search of your data in AGOL
    "type": "GeoJson" # define the type of data you are trying to publish
}

# URL to the GeoJSON file
geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
geojson_item = gis.content.add(item_properties, geojson_url)
 
# Publish the GeoJSON as a feature layer
BikeStationZone_data = geojson_item.publish()

print ('Data Published')

Data Published


In [5]:
# Dublin bike locations are available as a live dataset, updated every 5 minutes
# pull in the Dublin bike locations from the API

# API endpoint for bike locations
bike_url = "https://data.smartdublin.ie/bleeperbike-api/last_snapshot/"

# make a request to the API
response = requests.get(bike_url)

# if the request was successful (status code 201), get the data
if response.status_code == 201:
    # Parse the JSON response
    bikeData = response.json()
    
print ('success')

success


In [9]:
# Convert bike data to a feature layer collection
FL_collection = FeatureLayerCollection.fromitem(bikeData)

# Define item properties
item_properties = {
    "title": "Dublin Bike Locations",
    "description": "Dublin bike locations obtained from Smart Dublin API",
    "tags": "Dublin, Bike, Locations",
}

# Publish feature layer collection to ArcGIS Online
published_item = FL_collection.manager.publish(
    overwrite=True,
    tags=item_properties["tags"],
    description=item_properties["description"],
    thumbnail=None  # You can provide a thumbnail image if needed
)

# Share the feature layer collection
published_item.share(org=True)  # Share with your organization

print("Feature service created successfully:", published_item)


AttributeError: 'list' object has no attribute 'type'

In [ ]:
# convert JSON data to a dataframe
# bike_df = pd.DataFrame(bikeData)

In [ ]:
# show the first rows of data
# bike_df.head

In [61]:
try:
    # set up the feature service properties in AGOL for the bike locations dataset
    item_properties = {
        "title": "Dublin Bike Locations",
        "description": "Dublin bike locations obtained from Smart Dublin API",
        "tags": "Dublin, Bike, Locations",
    }

    feature_layer_collection = gis.content.create_from_xy(bike_data["features"], item_properties=item_properties)

    # Share the feature layer collection
    feature_layer_collection.share(org=True)

    print("Feature service created successfully:", feature_layer_collection)
except Exception as e:
    print("Failed to create feature service:", e)


Failed to create feature service: 'ContentManager' object has no attribute 'create_from_xy'
